In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from data.linear.binary import BinaryLinearData
from data.linear.continuous import GaussianLinearData
from algorithm.general import GConfounderTest, PearsonConfounderTest

from experiment.utils import run, save_results
from experiment.plot import plot_vary_conf, set_mpl_default_settings

set_mpl_default_settings()

# Experiment: Vary effect size of confounder

In [ ]:
load = False


# Data and algorithm
simulate_class = BinaryLinearData  
test_method = GConfounderTest()

# Experiment parameters

nbr_env = [i for i in range(100, 2100, 100)]
nbr_samples = [2]
repetitions = 500
sign_level = 0.05
vary_both_mechanisms = False # decide whether to vary confounder effect size to both T and Y, or only Y
compute_bias = False
conf_strength = list(np.linspace(0, 10.0, 20))
dist_param =  {'X': {'a': 0.0, 'b': 1},
               'Y': {'a': 0.0, 'b': 1},
               'T': {'a': 0.0, 'b': 1}
                    }

# Get timestamp for experiment
now = datetime.now()
timestamp = now.strftime("%m%d%H%M")
print('Timestamp:', timestamp)

In [ ]:
if not load: # run new experiment

    path = f'./results/vary_conf_{timestamp}.csv'
    out = run([dist_param, nbr_env, nbr_samples, conf_strength, simulate_class, test_method, repetitions, sign_level, compute_bias, vary_both_mechanisms], 
               save_during_run=path)
    experiment_results = pd.concat(out)

else: # load old experiment
    
    timestamp_str = "11222131"
    timestamp = int(timestamp_str)
    path = f'results/vary_conf_{timestamp_str}.csv'
    experiment_results = pd.read_csv(path)

# Plot results

In [ ]:
plot_vary_conf(experiment_results)
if not vary_both_mechanisms:
    plt.ylabel('Parameter $\lambda$')
else:
    plt.ylabel('Parameters $\lambda=\gamma$')

path = f'results/figures/vary_conf_{timestamp}.pdf'
plt.savefig(path, format='pdf', bbox_inches='tight')